In [1]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [1]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [4]:
# This is just a test
df = pd.DataFrame()
fixture = 'fixture.json'
team_home = dirName + '720/720.json'
team_away = dirName + '529/529.json'

with open(fixture) as f:
    my_fixture = json.load(f)
with open(team_home) as f:
    standings_home = json.load(f)
    del standings_home['stats_away']
    my_fixture.update({"team_home": standings_home})
with open(team_away) as f:
    standings_away = json.load(f)
    del standings_home['stats_home']
    my_fixture.update({"team_away": standings_away})
df = df.append(json_normalize(my_fixture))
df.head()

NameError: name 'dirName' is not defined

In [16]:
for col in df.columns: 
    print(col)

goals_home
goals_away
team_home.team.id
team_home.team.name
team_home.rank
team_home.points
team_home.goals_diff
team_home.form
team_home.description
team_away.team.id
team_away.team.name
team_away.rank
team_away.points
team_away.goals_diff
team_away.form
team_away.description
team_away.stats_home.played
team_away.stats_home.wins
team_away.stats_home.draws
team_away.stats_home.lose
team_away.stats_home.goals_for
team_away.stats_home.avg_goals_for
team_away.stats_home.goals_against
team_away.stats_home.avg_goals_against
team_away.stats_home.clean_sheet
team_away.stats_home.failed_to_score
team_away.stats_home.streaks.wins
team_away.stats_home.streaks.draws
team_away.stats_home.streaks.loses
team_away.stats_home.streaks.best_win
team_away.stats_home.streaks.best_lose
team_away.stats_home.streaks.best_goals_for
team_away.stats_home.streaks.best_goals_against
team_away.stats_away.played
team_away.stats_away.wins
team_away.stats_away.draws
team_away.stats_away.lose
team_away.stats_away.goal

In [2]:
league = '262'
season = '2020'

In [3]:
# League directory
dirName = league + '/' + season + '/' 

In [4]:
# Historicity: multi season directory
year_window = 1
start_season = 2020
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2020]


In [5]:
# Multi League directory
country = 'MX'
season = '2020'
dirCountry = '../' + country + '/'

In [6]:
# The last week played
curr_week = 7

In [20]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings to week: team : stands
visited_standings = {}
# Easy access to last standings
current_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if league == "teams.json":
        continue
    if ".csv" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Initializes all current standings
            if team_id not in current_standings:
                json_standings = dirTeam + team_id + '_' + str(curr_week) + '.json'
                with open(json_standings) as f:
                    curr_standings = json.load(f)
                    current_standings[team_id] = curr_standings
            else:
                curr_standings = current_standings[team_id]
            
            # Gets standings from a team if they have not being added
            for week in list(range(1, 39, 1)):
                week = str(week)
                if week not in visited_standings:
                    visited_standings[week] = {}
                    if team_id not in visited_standings[week]:
                        try:
                            json_standings = dirTeam + team_id + '_' + week + '.json'
                            with open(json_standings) as f:
                                standings = json.load(f)
                                visited_standings[week][team_id] = standings
                        except:
                            visited_standings[week][team_id] = curr_standings
                            standings = curr_standings

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        w = fixture["week"]
                        w = ''.join(filter(str.isdigit, w))
                        # Gets standings from away_team
                        if away_team in visited_standings[w]:
                            away_team_stands = visited_standings[w][away_team]
                        else:
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + week + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                            except:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(curr_week) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                                    current_standings[away_team] = away_team_stands
                        # Home team standings
                        if team_id in visited_standings[w]:
                            standings = visited_standings[w][team_id]
                        else:
                            standings = current_standings[team_id]
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            away_team_stands = current_standings[away_team]
                            # Changes if game to be played
                            standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings[w]:
                                away_team_stands = visited_standings[w][away_team]
                            else:
                                away_team_stands = current_standings[away_team]
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

In [21]:
df.head(10)

,id,season,week,goals_home,goals_away,league,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,569686,2020,Apertura - 4,1,0,262,2288,Necaxa,16,4,...,2,63%,2,None,3,465.0,389.0,84%,NaN,NaN
0,569777,2020,Apertura - 14,None,None,262,2288,Necaxa,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569724,2020,Apertura - 8,None,None,262,2288,Necaxa,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569795,2020,Apertura - 16,None,None,262,2288,Necaxa,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569679,2020,Apertura - 3,1,1,262,2288,Necaxa,18,1,...,None,43%,2,None,2,264.0,197.0,75%,NaN,NaN
0,569705,2020,Apertura - 6,2,1,262,2288,Necaxa,10,8,...,1,46%,None,None,3,258.0,181.0,70%,NaN,NaN
0,569740,2020,Apertura - 10,None,None,262,2288,Necaxa,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,570962,2020,Apertura - 1,0,3,262,2288,Necaxa,12,8,...,1,60%,1,None,1,547.0,474.0,87%,NaN,NaN
0,569749,2020,Apertura - 11,None,None,262,2288,Necaxa,12,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
0,569782,2020,Apertura - 14,None,None,262,2286,U.N.A.M. - Pumas,3,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [8]:
just_numbers = []
for fixture in visited_fixtures:
    just_numbers.append(int(fixture.strip('.json')))
print(just_numbers)

[5696864, 56977714, 5697248, 56979516, 5696793, 5697056, 56974010, 5709621, 56974911, 56978214, 56974510, 5697197, 5697268, 5696924, 5696843, 5696651, 56979916, 56976112, 5697329, 5696732, 56979015, 56975511, 5697005, 56980817, 5697106, 56977213, 56974710, 56979616, 56977413, 5697288, 5709651, 56975011, 5696783, 5696914, 5697046, 5697308, 5696944, 56974410, 56978915, 5697096, 56975411, 5696702, 56980116, 56977113, 56977513, 56975711, 5697217, 5697369, 56978414, 5697035, 5696762, 56980216, 56975611, 5709611, 56977614, 5697116, 56979416, 56976312, 5697319, 5697167, 5696853, 56979315, 56976612, 5696773, 5697238, 56981117, 5697126, 5696671, 56974810, 5696874, 56976512, 56980717, 5696641, 56978114, 5696813, 5696975, 5697379, 5697187, 5696803, 5709631, 5696995, 56975111, 56977814, 5697399, 56978615, 5697147, 5696965, 56980517, 5697349, 5697177, 56978014, 56979115, 5696833, 56976012, 5709641, 5696661, 5696985, 5696823, 5697157, 56980016, 56976212, 5697339, 56977914, 56978314, 56976412, 569792

In [22]:
df.shape

(153, 124)

In [23]:
# csv to save
file_title = 'ligaMX_7.csv'
csv_file = dirCountry + file_title

In [24]:
df.to_csv(csv_file)

In [51]:
os.listdir(dirName)

['533',
 '798',
 '532',
 '.DS_Store',
 '543',
 '715',
 '529',
 '545',
 '542',
 '537',
 '530',
 '539',
 '538',
 '531',
 '536',
 '540',
 '541',
 '546',
 '548',
 '720',
 '727']